In [ ]:
#!/usr/bin/env python
# coding: utf-8


In [ ]:
# <a href="https://colab.research.google.com/github/junghoum/Hello-world/blob/main/Untitled41.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# In[1]:


In [ ]:

import requests
from urllib import parse
from bs4 import BeautifulSoup
base_url = 'http://movie.naver.com/movie/point/af/list.nhn?&page={}'
url = base_url.format(1)
res = requests.get(url)


In [ ]:
if res.status_code == 200:
  soup = BeautifulSoup(res.text)
  tds = soup.select('table.list_netizen > tbody > tr > td.title')
  print(len(tds))
  for td in tds:
    movie_title = td.select_one('a.movie').text.strip()
    link = td.select_one('a.movie').get('href')
    link = parse.urljoin(base_url, link)
    score = td.select_one('div.list_netizen_score > em').text.strip()
    comment = td.select_one('br').next_sibling.strip()
    print(movie_title, link, score, comment, sep=' :: ')
    print('-------------------------------------------------')


In [ ]:

# In[2]:


In [ ]:

import random
random.uniform(0.2, 1.2)


In [ ]:

# In[3]:


In [ ]:

import requests
import time
import random
from bs4 import BeautifulSoup


In [ ]:

base_url = 'https://movie.naver.com/movie/point/af/list.nhn?&page={}'
#결과 저장할 리스트
comment_list = []
for page in range(1, 101):
    url = base_url.format(page)
    res=requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        tds = soup.select('table.list_netizen > tbody > tr > td.title')
        for td in tds:
            movie_title = td.select_one('a.movie').text.strip()
            link = td.select_one('a.movie').get('href')
            link = parse.urljoin(base_url, link)
            score = td.select_one('div.list_netizen_score > em').text.strip()
            comment = td.select_one('br').next_sibling.strip()
            # 리스트에 저장
            comment_list.append((movie_title, link, score, comment))
        interval = round(random.uniform(0.2, 1.2), 2)
        time.sleep(interval)
print('종료') 


In [ ]:

# In[4]:


In [ ]:

import pandas as pd
df = pd.DataFrame(comment_list, 
                  columns=['영화제목','영화링크', '평점','댓글'])
df.to_csv('naver_comment.csv', encoding='utf-8', index=False)


In [ ]:

# In[5]:


In [ ]:

df.head()


In [ ]:

# In[6]:


In [ ]:

df.shape


In [ ]:

# In[7]:


In [ ]:

get_ipython().system('pip install konlpy')


In [ ]:

# In[8]:


In [ ]:

import pandas as pd
import numpy as np


In [ ]:
from konlpy.tag import Okt
import re


In [ ]:
okt = Okt()
tagset_df = pd.Series(okt.tagset)
tagset_df


In [ ]:

# In[9]:


In [ ]:

def get_pos(x):
  tagger = Okt()
  pos = tagger.pos(x)


In [ ]:
  result = []


In [ ]:
  for al in pos:
    result.append(f'{al[0]}/{al[1]}')


In [ ]:
  return result


In [ ]:

# In[10]:


In [ ]:

get_ipython().run_line_magic('matplotlib', 'inline')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# In[11]:


In [ ]:

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result


In [ ]:

a = apply_regular_expression(df['댓글'][0])


In [ ]:

print(get_pos(a))


In [ ]:

# In[12]:


In [ ]:

df['댓글'][0]


In [ ]:

# In[13]:


In [ ]:

nouns = okt.nouns(apply_regular_expression(df['댓글'][0]))
nouns


In [ ]:

# In[14]:


In [ ]:

#말뭉치 생성


In [ ]:
corpus = "".join(df['댓글'].tolist())
corpus


In [ ]:

# In[15]:


In [ ]:

# 정규 표현식 적용


In [ ]:
apply_regular_expression(corpus)


In [ ]:

# In[16]:


In [ ]:

# 전체 말뭉치(corpus)에서 명사 형태소 추출


In [ ]:
nouns = okt.nouns(apply_regular_expression(corpus))
print(nouns)


In [ ]:

# In[17]:


In [ ]:

# 빈도 탐색
from collections import Counter


In [ ]:
counter = Counter(nouns)


In [ ]:

# In[18]:


In [ ]:

# 가장 많은 단어 top 10


In [ ]:
counter.most_common(10)


In [ ]:

# In[19]:


In [ ]:

# 한글자 명사 제거하고 top 10


In [ ]:
available_counter = Counter({x: counter[x] for x in counter if len(x) > 1})
available_counter.most_common(10)


In [ ]:

# In[20]:


In [ ]:

# "우리", "매우"와 같은 실질적인 의미가 없고 꾸밈 역할을 하는 불용어를 불용어 사전을 정의하여 제거


In [ ]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]


In [ ]:

# In[21]:


In [ ]:

# 위 단어들 외에도 분석하고자 하는 데이터셋에 특화된 불용어 단어들도 불용어 사전에 추가하기


In [ ]:
jeju_hotel_stopwords = ['영화', '리뷰']


In [ ]:
for word in jeju_hotel_stopwords:
  stopwords.append(word)


In [ ]:

# In[22]:


In [ ]:

# BoW 벡터 생성


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
def text_cleaning(text):
  hangul = re.compile('[^ ㄱ-ㅣ 가-힣]') # 정규 표현식 처리
  result = hangul.sub('', text)
  okt = Okt() #형태소 추출
  nouns = okt.nouns(result)
  nouns = [x for x in nouns if len(x) > 1] # 한글자 키워드 제거
  nouns = [x for x in nouns if x not in stopwords] # 불용어 제거
  return nouns


In [ ]:
vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(df['댓글'].tolist())
word_list = vect.get_feature_names()
count_list = bow_vect.toarray().sum(axis=0)


In [ ]:

# In[23]:


In [ ]:

# 단어 리스트


In [ ]:
word_list


In [ ]:

# In[24]:


In [ ]:

# 각 단어가 전체 리뷰중에 등장한 총 횟수


In [ ]:
count_list


In [ ]:

# In[25]:


In [ ]:

# 각 단어의 리뷰별 등장 횟수


In [ ]:
bow_vect.toarray()


In [ ]:

# In[26]:


In [ ]:

bow_vect.shape


In [ ]:

# In[27]:


In [ ]:

# "단어" - "총 등장 횟수" Matching


In [ ]:
word_count_dict = dict(zip(word_list, count_list))


In [ ]:
word_count_dict


In [ ]:

# In[28]:


In [ ]:

# TF-IDF 적용(변환)


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer


In [ ]:
tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)


In [ ]:

# In[29]:


In [ ]:

print(tf_idf_vect.shape)


In [ ]:

# In[30]:


In [ ]:

# 변화 후 1000*3599 matrix가 출력된다. 여기서 한 행은 한 리뷰를 의미라고, 한 열은 한 단어를 의미한다.


In [ ]:
# 첫 번째 리뷰에서의 단어 중요도(TF-IDF 값) -- 0이 아닌 것만 출력


In [ ]:
print(tf_idf_vect[0])


In [ ]:

# In[31]:


In [ ]:

# 첫 번째 리뷰에서 모든 단어의 중요도 -- 0인 값까지 포함


In [ ]:
print(tf_idf_vect[0].toarray().shape)
print(tf_idf_vect[0].toarray())


In [ ]:

# In[32]:


In [ ]:

# "벡터", "단어" mapping


In [ ]:
vect.vocabulary_


In [ ]:

# In[33]:


In [ ]:

invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
print(str(invert_index_vectorizer)[:100]+'...')


In [ ]:

# In[34]:


In [ ]:

# 감성 분류 -- Logistic Regression


In [ ]:

# In[35]:


In [ ]:

# 데이터셋 생성, 우리는 이용자의 리뷰를 "긍정"/"부정" 두가지 부류로 나누고자 한다.하지만 이러한 이용자의 감성을 대표할 수 있는 "평가 점수" 변수는 1~5의  value를 가지고 있다. 따라서 "평가 점수"변수 (rating: 1~5)를 이진 변수(긍정:1 부정:0)으로 변환해야 한다.


In [ ]:
df.sample(10)


In [ ]:

# In[36]:


In [ ]:

df['평점'] = df['평점'].astype(str).astype(int)


In [ ]:

# In[37]:


In [ ]:

# 리뷰 내용과 평점을 보면 4~5 는 긍정적인 리뷰, 1~3은 부정적인 리뷰가 많이 보였기에 4~5는 1, 1~3은 0 으로 분류


In [ ]:
df['평점'].hist()


In [ ]:
a = df['평점']


In [ ]:

# In[38]:


In [ ]:

def rating_to_label(a):
  if a > 7:
    return 1
  else:
    return 0


In [ ]:
df['y'] = a.apply(lambda x: rating_to_label(x))


In [ ]:

# In[39]:


In [ ]:

df.head()


In [ ]:

# In[40]:


In [ ]:

# Training set/ Test set 나누기


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x = tf_idf_vect
y = df['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=2)


In [ ]:

# In[41]:


In [ ]:

# model 학습하기


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
# fit in training set
lr = LogisticRegression(random_state = 0)
lr.fit(x_train, y_train)


In [ ]:
# predict in test set
y_pred = lr.predict(x_test)


In [ ]:

# In[42]:


In [ ]:

# 분류 결고 평가


In [ ]:
# classification result for test set


In [ ]:
# WARNING: Multi Test Leakage — test data is being used improperly before final evaluation.
# QUICK FIX: Use a proper Train/Validation/Test split to ensure test data is untouched until final evaluation.
print('accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('precision: %.2f' % precision_score(y_test, y_pred))
print('recall: %.2f' % recall_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))


In [ ]:

# In[43]:


In [ ]:

# confusion matrix


In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
confu = confusion_matrix(y_true = y_test, y_pred = y_pred)


In [ ]:
plt.figure(figsize=(4, 3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# 모델 평가결과를 살펴보면, 모델이 지나치게 긍정("1")으로만 예측하는 경향이 있습니다. 따라서 긍정 리뷰를 잘 예측하지만, 부정 리뷰에 대한 예측 정확도가 매우 낮다. 이는 샘플데이터의 클래스 불균형으로 인한 문제로 보인다.
# 따라서, 클래스 불균형 조정을 진행


In [ ]:

# In[44]:


In [ ]:

# 샘플링 재조정하기


In [ ]:
# 1:1 Sampling


In [ ]:
df['y'].value_counts()


In [ ]:

# In[45]:


In [ ]:

positive_random_idx = df[df['y']==1].sample(275, random_state=12).index.tolist()
negative_random_idx = df[df['y']==0].sample(275, random_state=12).index.tolist()


In [ ]:
# In[46]:


In [ ]:

random_idx = positive_random_idx + negative_random_idx
x = tf_idf_vect[random_idx]
y = df['y'][random_idx]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 2)


In [ ]:

# In[47]:


In [ ]:

# 모델 재학습


In [ ]:
lr2 = LogisticRegression(random_state = 0)
lr2.fit(x_train, y_train)
y_pred = lr2.predict(x_test)


In [ ]:

# In[48]:


In [ ]:

#clasification result for test set


In [ ]:
print('accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('precision: %.2f' % precision_score(y_test, y_pred))
print('recall: %.2f' % recall_score(y_test, y_pred))
print('F1: %.2f' % f1_score(y_test, y_pred))


In [ ]:

# In[49]:


In [ ]:

# confusion matrix


In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
confu = confusion_matrix(y_true = y_test, y_pred = y_pred)


In [ ]:
plt.figure(figsize=(4,3))
sns.heatmap(confu, annot=True, annot_kws={'size':15}, cmap='OrRd', fmt='.10g')
plt.show()


In [ ]:
# 이제 모델이 "긍정적인" 케이스와 "부정적인"케이스를 모두 적당히 잘 맞춘 것을 확인할 수 있습니다.


In [ ]:

# In[50]:


In [ ]:

# 긍정/ 부정 키원드 분석


In [ ]:
# 긍정/ 부정 키원드를 추출하기 위해 먼저 Logistic Regression 모델에 각 단어의
# coeficient를 시각화해보기


In [ ]:
lr2.coef_


In [ ]:

# In[51]:


In [ ]:

# print logistic regression's coef


In [ ]:
plt.figure(figsize=(10, 8))
plt.bar(range(len(lr2.coef_[0])), lr2.coef_[0])


In [ ]:
# 여기서 계수가 양인 경우는 단어가 긍정적인 영향을 미쳤다고 볼 수 있고, 반면에, 음인 경우는 부정적인 영향을 미쳤다고 볼 수 있습니다.
# 이 계수들을 크기순으로 정렬하면, 긍정/ 부정 키워드를 출력하는 지표가 되겠습니다.


In [ ]:

# In[52]:


In [ ]:

# 긍정 키원드와 부정 키워드의 top 5를 각각 출력해보기


In [ ]:
print(sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)[:5])


In [ ]:
print(sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse = True)[-5:])


In [ ]:
# 단어의 coeficient와 index가 출력된다.


In [ ]:

# In[53]:


In [ ]:

# 전체 단어가 포함한 "긍정 키원드 리스트"와 "부정 키원드 리스트"를 정의하고 출력하기


In [ ]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse=True)


In [ ]:
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr2.coef_[0])), reverse=False)


In [ ]:
coef_pos_index


In [ ]:

# In[54]:


In [ ]:

invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
invert_index_vectorizer


In [ ]:

# In[55]:


In [ ]:

for coef in coef_pos_index[:20]:
  print(invert_index_vectorizer[coef[1]], coef[0])


In [ ]:

# In[56]:


In [ ]:

for coef in coef_neg_index[:20]:
  print(invert_index_vectorizer[coef[1]], coef[0])


In [ ]:

# In[56]:
